In [1]:
from utils.get_dataset_info import get_dataset_info
import cv2 as cv
import matplotlib.pyplot as plt
import numpy as np
from utils.get_dataset_info import get_dataset_info
import cv2 as cv
import numpy as np
from utils import general, pflat, estimate_T_robust, levenberg_marquardt, triangulate_3D_point_DLT, viz
from romatch import roma_outdoor
import torch
import os
from tqdm import tqdm

K, img_names, init_pair, pixel_threshold = get_dataset_info(3)
K_inv = np.linalg.inv(K)
image_paths = ['./project_data/'+img_name for img_name in img_names]

In [2]:
 # Normalize pixel threshold
focal_length = (K[0,0] + K[1 ,1])/2 # Average over the diagonal
epipolar_treshold =          pixel_threshold / focal_length
homography_threshold =   3 * pixel_threshold / focal_length
translation_threshold  = 3 * pixel_threshold / focal_length

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
roma_model = roma_outdoor(device=device)

In [ ]:
relative_rotations = general.find_relative_rotation(image_paths, K_inv, roma_model, epipolar_treshold, device=device)

In [5]:
absolute_rotations = general.upgrade_to_absolute_rotation(relative_rotations)

In [11]:
def estimate_T_robust(xs, Xs, R, eps, iterations=100):
    assert xs.shape[-1] == Xs.shape[-1]
    assert xs.shape[0] == 3
    assert Xs.shape[0] == 3
    
    best_num_inliers = 0
    best_T = None
    for i in tqdm(range(iterations), desc="RANSAC iterations"):
        randind = np.random.choice(xs.shape[1], size=2, replace=False)
        x, X = xs[:, randind], Xs[:, randind]
        M = np.zeros((6, 4))

        for j in range(2):  # Two correspondences
            Xj = X[:, j]
            xj = x[:, j]
            M[3*j + 0] = [1, 0, 0, -xj[0]+R[0, :]@Xj]
            M[3*j + 1] = [0, 1, 0, -xj[1]+R[1, :]@Xj]
            M[3*j + 2] = [0, 0, 1, -xj[2]+R[2, :]@Xj]
        _, S, Vt = np.linalg.svd(M)
        T = Vt[-1, :3].reshape(-1, 1)  # The translation is the last column of Vt
        print('Smallest singular value: ', S[-1])
        
        projected_points = R @ Xs[:3, :] + T  # 3 x N
        projected_points /= projected_points[2, :]  # Normalize to homogeneous (2D)
        errors = np.linalg.norm(xs[:2, :] - projected_points[:2, :], axis=0)**2
        inliers = errors < eps**2
        num_inliers = np.sum(inliers)
        
        if num_inliers > best_num_inliers:
            best_num_inliers = num_inliers
            best_T = T
    print(f"After running RANSAC for the reduced camera resectioning problem for {iterations} iterations the best solution has {best_num_inliers} inliers of {xs.shape[-1]} points. I.e. c. {best_num_inliers/xs.shape[-1]*100:.2f}%")
    return best_T
    

In [12]:
T = estimate_T_robust(x2n, X, R, translation_threshold, iterations=100)

RANSAC iterations: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 3738.54it/s]

Smallest singular value:  0.4140936165730035
Smallest singular value:  0.4686381106983542
Smallest singular value:  0.378188087506088
Smallest singular value:  0.23112476749696975
Smallest singular value:  0.2008787014954546
Smallest singular value:  0.4549941763128682
Smallest singular value:  0.3436488990352502
Smallest singular value:  0.18257859480055344
Smallest singular value:  0.21292327413999337
Smallest singular value:  0.283429791774613
Smallest singular value:  0.2860381211433134
Smallest singular value:  0.34277952035001585
Smallest singular value:  0.17364635949520682
Smallest singular value:  0.36015932135505757
Smallest singular value:  0.1647438115407505
Smallest singular value:  0.19793720217994004
Smallest singular value:  0.22482313922001113
Smallest singular value:  0.1102176278730449
Smallest singular value:  0.3520171201752913
Smallest singular value:  0.3098060087036482
Smallest singular value:  0.4055424719384894
Smallest singular value:  0.06712846020383953
Sma

In [9]:
imA_path = image_paths[init_pair[0]-1]
Ps = []
Xs = []
xs = []
for i in range(len(image_paths)):
    R = absolute_rotations[i]
    if i == init_pair[0]-1:
        print('Skipping...')
        T=np.zeros((3,1))
    else:
        x1u, x2u = general.find_matches(imA_path, image_paths[i], roma_model, device=device)
        x1n = pflat.pflat(K_inv @ x1u)
        x2n = pflat.pflat(K_inv @ x2u)
        X = general.triangulate_initial_points(x1n, x2n, epipolar_treshold)
        X = (absolute_rotations[init_pair[0]-1]).T@X.T
        
        T = estimate_T_robust(x2n, X, R, translation_threshold, iterations=100)
        break

RANSAC iterations: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:07<00:00, 13.56it/s]


After running RANSAC for 100 iterations the best solution has 769 inliers of 10000 points. I.e. c. 7.69%


RANSAC iterations: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 4509.76it/s]

After running RANSAC for the reduced camera resectioning problem for 100 iterations the best solution has 0 inliers of 10000 points. I.e. c. 0.00%


In [ ]:
imA_path = image_paths[init_pair[0]-1]
Ps = []
Xs = []
xs = []
for i in range(len(image_paths)):
    R = absolute_rotations[i]
    if i == init_pair[0]-1:
        print('Skipping...')
        T=np.zeros((3,1))
    else:
        x1u, x2u = general.find_matches(imA_path, image_paths[i], roma_model, device=device)
        x1n = pflat.pflat(K_inv @ x1u)
        x2n = pflat.pflat(K_inv @ x2u)
        X = general.triangulate_initial_points(x1n, x2n, epipolar_treshold)
        X = (absolute_rotations[init_pair[0]-1]).T@X.T
        # T = estimate_T_robust(x2n, X, R, translation_threshold)
        rvec, _ = cv.Rodrigues(R)
        success, rvec, T, inliers = cv.solvePnPRansac(X.T, x2u[:2].T, K, None, rvec=rvec, useExtrinsicGuess=True)
        Ps.append(np.hstack((R, T)))
        Xs.append(X)
        xs.append(x2n)

viz.plot_scene(X.T, Ps, name=f"test.png")

In [ ]:
Xs = [general.make_homogenous(Xi.T) for Xi in Xs]
viz.plot_scene(Xs[0], Ps, name=f"before_BA.png")
Ps = levenberg_marquardt.perform_bundle_adjustment(Xs, xs, Ps)

In [ ]:
viz.plot_scene(Xs[0], Ps, name=f"after_BA.png")